# PPO

In [1]:
import logging
logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)  # set level to INFO for wordy
import matplotlib.pyplot as plt
from IPython.display import HTML

import numpy as np
import jax.numpy as jnp

from extravaganza.dynamical_systems import PPOGym

from extravaganza.lifters import NoLift, RandomLift, LearnedLift
from extravaganza.sysid import SysID
from extravaganza.controllers import LiftedBPC, ConstantController
from extravaganza.rescalers import ADAM, D_ADAM, DoWG
from extravaganza.utils import ylim, render
from extravaganza.experiments import Experiment

# seeds for randomness. setting to `None` uses random seeds
SYSTEM_SEED = None
CONTROLLER_SEED = None
LIFTER_AND_SYSID_SEED = None

## System
Here, we tune the parameters of a PPO algorithm learning on an OpenAI Gym environment. The PPO update step is
$$L^{CLIP}(\theta) = \mathbb{E} \left[ \min\left(r(\theta, \theta_{\text{old}}, s_t, a_t) \cdot \hat{A}_t, \text{clip}\left(r(\theta, \theta_{\text{old}}, s_t, a_t), 1 - \epsilon, 1 + \epsilon\right) \cdot \hat{A}_t\right)\right],
$$
and so we can tune the clip parameter $\epsilon$ in addition to learning rate $\eta$ and more.

We play on games like `CartPole`.

## Hyperparameters

In [2]:
name = 'ppo_????'
filename = '../logs/{}.pkl'.format(name)

def get_experiment_args():
    # --------------------------------------------------------------------------------------
    # ------------------------    EXPERIMENT HYPERPARAMETERS    ----------------------------
    # --------------------------------------------------------------------------------------

    num_trials = 1
    T = 3000
    T0 = 500000
    reset_condition = lambda t: False
    use_multiprocessing = False
    render_every = None

    # num_trials = 5
    # T = 12000  # total timesteps
    # T0 = 8000  # number of timesteps to just sysid for our methods
    # reset_condition = lambda t: t <= 10000 and t % 1000 == 0  # when to reset the system
    # use_multiprocessing = False
    # render_every = None

    # --------------------------------------------------------------------------------------
    # --------------------------    SYSTEM HYPERPARAMETERS    ------------------------------
    # --------------------------------------------------------------------------------------

    du = 1  # control dim
    initial_eps_clip = 0.1  # initial value for eps
    base = 1.5
    def apply_control(control, system): system.ppo.eps_clip = initial_eps_clip * 1.5 ** np.clip(control.item(), -7, 7)

    env_name = 'CartPole-v1'  
    # env_name = 'MountainCarContinuous-v0'
    continuous_action_space = False

    system_args = {
        'apply_control': apply_control,
        'control_dim': du,
        'env_name': env_name,
        'lr_actor': 0.0003,
        'lr_critic': 0.001,
        'eps_clip': initial_eps_clip,
        'gamma': 0.98,
        'max_episode_len': 400,
        'repeat': 10,
        'gym_repeat': 1,
        'seed': SYSTEM_SEED,
    }
    make_system = lambda : PPOGym(**system_args)

    # --------------------------------------------------------------------------------------
    # ------------------------    LIFT/SYSID HYPERPARAMETERS    ----------------------------
    # --------------------------------------------------------------------------------------

    sysid_method = 'regression'
    sysid_scale = 0.05

    learned_lift_args = {
        'lift_lr': 0.004,
        'sysid_lr': 0.004,
        'cost_lr': 0.001,
        'depth': 4,
        'buffer_maxlen': int(1e6),
        'num_epochs': 100,
        'batch_size': 64,
        'seed': LIFTER_AND_SYSID_SEED
    }

    # --------------------------------------------------------------------------------------
    # ------------------------    CONTROLLER HYPERPARAMETERS    ----------------------------
    # --------------------------------------------------------------------------------------

    h = 10  # controller memory length (# of w's to use on inference)
    hh = 10  # history length of the cost/control histories
    lift_dim = 20  # dimension to lift to

    m_update_rescaler = lambda : ADAM(0.002, betas=(0.9, 0.999))
    m0_update_rescaler = lambda : ADAM(0.00, betas=(0.9, 0.999))
    k_update_rescaler = lambda : ADAM(0.002, betas=(0.9, 0.999))

    lifted_bpc_args = {
        'h': h,
        'method': 'REINFORCE',
        'initial_scales': (0.01, 0.01, 0.01),  # M, M0, K   (uses M0's scale for REINFORCE)
        'rescalers': (m_update_rescaler, m0_update_rescaler, k_update_rescaler),
        'T0': T0,
    #     'bounds': (-5, 5),
        'initial_u': jnp.zeros(du),
        'decay_scales': False,
        'use_tanh': False,
        'use_K_from_sysid': False,
        'seed': CONTROLLER_SEED
    }

    make_controllers = {
        # these are w.r.t initial value of 0.1 with base of 1.5
#         '0.01': lambda sys: ConstantController(-5.67887358, du),
        '0.1': lambda sys: ConstantController(0, du),
        '0.2': lambda sys: ConstantController(1.71, du),
        '0.9': lambda sys: ConstantController(5.419, du),

    #     'No Lift': lambda sys: LiftedBPC(lifter=NoLift(hh, du, LIFTER_AND_SYSID_SEED), sysid=SysID(sysid_method, du, hh, sysid_scale, LIFTER_AND_SYSID_SEED), **lifted_bpc_args),
    #     'Random Lift': lambda sys: LiftedBPC(lifter=RandomLift(hh, du, lift_dim, learned_lift_args['depth'], LIFTER_AND_SYSID_SEED), sysid=SysID(sysid_method, du, lift_dim, sysid_scale, LIFTER_AND_SYSID_SEED), **lifted_bpc_args),
    #     'Learned Lift': lambda sys: LiftedBPC(lifter=LearnedLift(hh, du, lift_dim, scale=sysid_scale, **learned_lift_args), **lifted_bpc_args)
    }
    experiment_args = {
        'make_system': make_system,
        'make_controllers': make_controllers,
        'num_trials': num_trials,
        'T': T,
        'reset_condition': reset_condition,
        'reset_seed': SYSTEM_SEED,
        'use_multiprocessing': use_multiprocessing,
        'render_every': render_every,
    }
    return experiment_args

## actually run the thing :)

In [3]:
# run
experiment = Experiment(name)
stats = experiment(get_experiment_args)

INFO: Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO: Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO: Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
INFO: Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.
INFO: (EXPERIMENT) --------------------------------------------------
INFO: (EXPERIMENT) ----------------- TRIAL 0 -----------------------
INFO: (EXPERIMENT) --------------------------------------------------

INFO: (EXPERIMENT): testing 0.1
  0%|▎                                                                                 | 13/3000 [00:00<00:46, 63.61it/s, control=0, cost=-23]

tensor([ 0.0412,  0.3955, -0.0358, -0.6127]) tensor([ 0.0491,  0.5911, -0.0480, -0.9164]) False 1.0
tensor([ 0.0491,  0.5911, -0.0480, -0.9164]) tensor([ 0.0609,  0.7869, -0.0664, -1.2238]) False 1.0
tensor([ 0.0609,  0.7869, -0.0664, -1.2238]) tensor([ 0.0767,  0.5926, -0.0908, -0.9527]) False 1.0
tensor([ 0.0767,  0.5926, -0.0908, -0.9527]) tensor([ 0.0885,  0.7889, -0.1099, -1.2724]) False 1.0
tensor([ 0.0885,  0.7889, -0.1099, -1.2724]) tensor([ 0.1043,  0.5953, -0.1353, -1.0161]) False 1.0
tensor([ 0.1043,  0.5953, -0.1353, -1.0161]) tensor([ 0.1162,  0.7919, -0.1557, -1.3480]) False 1.0
tensor([ 0.1162,  0.7919, -0.1557, -1.3480]) tensor([ 0.1320,  0.5991, -0.1826, -1.1078]) False 1.0
tensor([ 0.1320,  0.5991, -0.1826, -1.1078]) tensor([ 0.1440,  0.7961, -0.2048, -1.4518]) False 1.0
tensor([ 0.1440,  0.7961, -0.2048, -1.4518]) tensor([0., 0., 0., 0.]) True 1.0
tensor([ 0.0233,  0.4150,  0.0098, -0.5586]) tensor([ 0.0316,  0.6100, -0.0014, -0.8482]) False 1.0
tensor([ 0.0316,  0.6

  1%|▊                                                                                 | 30/3000 [00:00<00:39, 75.74it/s, control=0, cost=-10]

tensor([ 0.0198,  0.3520, -0.0179, -0.5511]) tensor([ 0.0269,  0.1571, -0.0289, -0.2641]) False 1.0
tensor([ 0.0269,  0.1571, -0.0289, -0.2641]) tensor([ 0.0300,  0.3527, -0.0342, -0.5658]) False 1.0
tensor([ 0.0300,  0.3527, -0.0342, -0.5658]) tensor([ 0.0371,  0.1580, -0.0455, -0.2841]) False 1.0
tensor([ 0.0371,  0.1580, -0.0455, -0.2841]) tensor([ 0.0402, -0.0364, -0.0512, -0.0061]) False 1.0
tensor([ 0.0402, -0.0364, -0.0512, -0.0061]) tensor([ 0.0395,  0.1594, -0.0513, -0.3145]) False 1.0
tensor([ 0.0395,  0.1594, -0.0513, -0.3145]) tensor([ 0.0427,  0.3552, -0.0576, -0.6229]) False 1.0
tensor([ 0.0427,  0.3552, -0.0576, -0.6229]) tensor([ 0.0498,  0.1610, -0.0701, -0.3489]) False 1.0
tensor([ 0.0498,  0.1610, -0.0701, -0.3489]) tensor([ 0.0530, -0.0331, -0.0771, -0.0791]) False 1.0
tensor([ 0.0530, -0.0331, -0.0771, -0.0791]) tensor([ 0.0524,  0.1630, -0.0786, -0.3951]) False 1.0
tensor([ 0.0524,  0.1630, -0.0786, -0.3951]) tensor([ 0.0556, -0.0309, -0.0865, -0.1282]) False 1.0


  2%|█▎                                                                                | 50/3000 [00:00<00:33, 88.35it/s, control=0, cost=-10]

...

 26%|█████████████████████▍                                                           | 794/3000 [00:12<00:40, 53.97it/s, control=0, cost=-10]

tensor([ 0.1015,  0.1749, -0.1190, -0.3979]) tensor([ 0.1050, -0.0184, -0.1270, -0.1450]) False 1.0
tensor([ 0.1050, -0.0184, -0.1270, -0.1450]) tensor([ 0.1046, -0.2115, -0.1299,  0.1051]) False 1.0
tensor([ 0.1046, -0.2115, -0.1299,  0.1051]) tensor([ 0.1004, -0.0148, -0.1278, -0.2256]) False 1.0
tensor([ 0.1004, -0.0148, -0.1278, -0.2256]) tensor([ 0.1001, -0.2078, -0.1323,  0.0242]) False 1.0
tensor([ 0.1001, -0.2078, -0.1323,  0.0242]) tensor([ 0.0959, -0.0111, -0.1318, -0.3071]) False 1.0
tensor([ 0.0959, -0.0111, -0.1318, -0.3071]) tensor([ 0.0957, -0.2041, -0.1380, -0.0588]) False 1.0
tensor([ 0.0957, -0.2041, -0.1380, -0.0588]) tensor([ 0.0916, -0.0073, -0.1391, -0.3916]) False 1.0
tensor([ 0.0916, -0.0073, -0.1391, -0.3916]) tensor([ 0.0915, -0.2002, -0.1470, -0.1458]) False 1.0
tensor([ 0.0915, -0.2002, -0.1470, -0.1458]) tensor([ 0.0875, -0.3930, -0.1499,  0.0971]) False 1.0
tensor([ 0.0875, -0.3930, -0.1499,  0.0971]) tensor([ 0.0796, -0.5857, -0.1479,  0.3390]) False 1.0


 27%|█████████████████████▊                                                           | 807/3000 [00:12<00:35, 62.33it/s, control=0, cost=-40]


tensor([0., 0., 0., 0.]) True 1.0
tensor([ 0.0496,  0.3640, -0.0513, -0.6333]) tensor([ 0.0569,  0.1696, -0.0639, -0.3571]) False 1.0
tensor([ 0.0569,  0.1696, -0.0639, -0.3571]) tensor([ 0.0603, -0.0246, -0.0711, -0.0853]) False 1.0
tensor([ 0.0603, -0.0246, -0.0711, -0.0853]) tensor([ 0.0598,  0.1715, -0.0728, -0.3995]) False 1.0
tensor([ 0.0598,  0.1715, -0.0728, -0.3995]) tensor([ 0.0632,  0.3676, -0.0808, -0.7142]) False 1.0
tensor([ 0.0632,  0.3676, -0.0808, -0.7142]) tensor([ 0.0705,  0.1737, -0.0951, -0.4480]) False 1.0
tensor([ 0.0705,  0.1737, -0.0951, -0.4480]) tensor([ 0.0740, -0.0200, -0.1040, -0.1868]) False 1.0
tensor([ 0.0740, -0.0200, -0.1040, -0.1868]) tensor([ 0.0736, -0.2135, -0.1078,  0.0714]) False 1.0
tensor([ 0.0736, -0.2135, -0.1078,  0.0714]) tensor([ 0.0694, -0.4069, -0.1063,  0.3282]) False 1.0
tensor([ 0.0694, -0.4069, -0.1063,  0.3282]) tensor([ 0.0612, -0.2105, -0.0998,  0.0040]) False 1.0
tensor([ 0.0612, -0.2105, -0.0998,  0.0040]) tensor([ 0.0570, -0.0


KeyboardInterrupt



In [ ]:
# # save args and stats!  --  note that to save the args, we actually save the `get_args` function. we can print the 
# #                           source code later to see the hyperparameters we chose
# experiment.save(filename)

## Visualization
We keep track of the useful information through `Stats` objects, which can `register()` a variable to keep track of (which it does via calls to `update()`) and which can be aggregated via `Stats.aggregate()` for mean and variance statistics. 

We define below a plotting arrangement that plots all the desired quantities from both the system and controller.

In [ ]:
def plot_ppo(experiment: Experiment):
    assert experiment.stats is not None, 'cannot plot the results of an experiment that hasnt been run'
    all_stats = experiment.stats
    
    # clear plot and calc nrows
    plt.clf()
    n = 5
    nrows = n + (len(all_stats) + 1) // 2
    fig, ax = plt.subplots(nrows, 2, figsize=(16, 4 * nrows))

    # plot system and controller stats
    for i, (method, stats) in enumerate(all_stats.items()):
        if stats is None: 
            print('WARNING: {} had no stats'.format(method))
            continue
        stats.plot(ax[0, 0], 'eps_clip', label=method)
#         stats.plot(ax[0, 1], 'ws', label=method)
        stats.plot(ax[1, 0], 'us', label=method)
        stats.plot(ax[2, 0], 'rewards', label=method)
        stats.plot(ax[2, 1], 'avg costs since reset', label=method)
    
        stats.plot(ax[3, 0], '||A||_op', label=method)
        stats.plot(ax[3, 1], '||B||_F', label=method)
        stats.plot(ax[4, 0], '||A-BK||_op', label=method)
        stats.plot(ax[4, 1], 'lifter losses', label=method)
        i_ax = ax[n + i // 2, i % 2]
        stats.plot(ax[0, 1], 'disturbances', label=method)
        stats.plot(i_ax, 'K @ state', label='K @ state')
        stats.plot(i_ax, 'M \cdot w', label='M \cdot w')
        stats.plot(i_ax, 'M0', label='M0')
        i_ax.set_title('u decomp for {}'.format(method))
        i_ax.legend()

    # set titles and legends and limits and such
    # (note: `ylim()` is so useful! because sometimes one thing blows up and then autoscale messes up all plots)
    _ax = ax[0, 0]; _ax.set_title('state'); _ax.legend(); ylim(_ax, -1, 2)
    _ax = ax[0, 1]; _ax.set_title('disturbances'); _ax.legend(); ylim(_ax, -5, 5)
    _ax = ax[1, 0]; _ax.set_title('controls'); _ax.legend(); ylim(_ax, -3, 3)
    
    _ax = ax[2, 0]; _ax.set_title('instantaneous rewards'); _ax.legend()
    _ax = ax[2, 1]; _ax.set_title('avg rewards since reset'); _ax.legend()    
    
    _ax = ax[3, 0]; _ax.set_title('||A||_op'); _ax.legend()
    _ax = ax[3, 1]; _ax.set_title('||B||_F'); _ax.legend()
    
    _ax = ax[4, 0]; _ax.set_title('||A-BK||_op'); _ax.legend()
    _ax = ax[4, 1]; _ax.set_title('lifter losses'); _ax.legend()
    pass

### Plot

In [ ]:
# plot_ppo(experiment)
print(experiment.stats['0.1']._stats['us'].ts)

### Dynamic Plot

In [ ]:
# # dynamic plot
# anim = render(experiment, 'xs', 'rewards', sliderkey='us', save_path=None, duration=5)
# vid = anim.to_html5_video()
# HTML(vid)